In [28]:
import pandas as pd
import numpy as np
import os
import glob
from itertools import combinations
from collections import defaultdict
from random import sample
import random
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_context('poster')
sns.set_theme(style="whitegrid")
#pandas max row

# Gilon Activity

In [29]:
def read_files(file_list):
    file_list.sort()
    print(f"Total files: {len(file_list)}")
    all_results = []
    for file in file_list:
        df = pd.read_csv(file)
        all_results.append(df)
    return pd.concat(all_results)

In [30]:
gilon_activity_main_dir = 'dataset/importance_result_files_mine/gilon_activity/'
shufflenet_activity = glob.glob(gilon_activity_main_dir + 'shufflenet/*')
resnet_activity = glob.glob(gilon_activity_main_dir + 'resnet9/*')
mlpmixer_activity = glob.glob(gilon_activity_main_dir + 'mlpmixer/*')
simplevit_activity = glob.glob(gilon_activity_main_dir + 'simplevit/*')

shufflenet_activity_df = read_files(shufflenet_activity)
resnet_activity_df = read_files(resnet_activity)
mlpmixer_activity_df = read_files(mlpmixer_activity)
simplevit_activity_df = read_files(simplevit_activity)

shufflenet_gilon_qr_true = shufflenet_activity_df[shufflenet_activity_df['opl_loss']==True]
shufflenet_gilon_qr_false = shufflenet_activity_df[shufflenet_activity_df['opl_loss']==False]
resnet_gilon_qr_true = resnet_activity_df[resnet_activity_df['opl_loss']==True]
resnet_gilon_qr_false = resnet_activity_df[resnet_activity_df['opl_loss']==False]
mlpmixer_gilon_qr_true = mlpmixer_activity_df[mlpmixer_activity_df['opl_loss']==True]
mlpmixer_gilon_qr_false = mlpmixer_activity_df[mlpmixer_activity_df['opl_loss']==False]
simplevit_gilon_qr_true = simplevit_activity_df[simplevit_activity_df['opl_loss']==True]
simplevit_gilon_qr_false = simplevit_activity_df[simplevit_activity_df['opl_loss']==False]

all_results = [[shufflenet_gilon_qr_false, 'ShuffleNet(CE)'],
                [shufflenet_gilon_qr_true, 'ShuffleNet(QR)'],
                [resnet_gilon_qr_false, 'ResNet9(CE)'],
                [resnet_gilon_qr_true, 'ResNet9(QR)'],
                [mlpmixer_gilon_qr_false, 'MLPMixer(CE)'],
                [mlpmixer_gilon_qr_true, 'MLPMixer(QR)'],
                [simplevit_gilon_qr_false, 'SimpleViT(CE)'],
                [simplevit_gilon_qr_true, 'SimpleViT(QR)']]

Total files: 6
Total files: 6
Total files: 6
Total files: 5


In [31]:
def calculate_drop_acc_test(df, drop_k=2):
    df = df[df['exp_type']=='important'].groupby('exp_num').mean()
    base_acc = df[df['feature_order']==0]['accuracy'].mean()
    k_acc = df[df['feature_order']==drop_k]['accuracy'].mean()

    base_loss = df[df['feature_order']==0]['test_loss'].mean()
    k_loss = df[df['feature_order']==drop_k]['test_loss'].mean()
    # calculate as percentage
    drop_acc = (base_acc - k_acc) / base_acc * 100
    drop_loss = (k_loss - base_loss) / base_loss * 100
    print(f"Drop {drop_k} accuracy: {drop_acc:.3f}")
    print(f"Drop {drop_k} loss: {drop_loss:.3f}")

    return [drop_acc, drop_loss, drop_k, base_acc, k_acc, base_loss, k_loss]

In [32]:
all_drop_results = []
for result in all_results:
    drop_results = calculate_drop_acc_test(result[0], drop_k=3)
    all_drop_results.append([result[1], drop_results[0], drop_results[1], drop_results[2], drop_results[3], drop_results[4], drop_results[5], drop_results[6]])
all_drop_results_df = pd.DataFrame(all_drop_results, columns=['model', 'drop_acc', 'increase_loss', 'drop_k', 'base_acc', 'k_acc', 'base_loss', 'k_loss'])
all_drop_results_df

Drop 3 accuracy: 3.326
Drop 3 loss: 22.894
Drop 3 accuracy: 8.158
Drop 3 loss: 95.767
Drop 3 accuracy: 1.553
Drop 3 loss: 36.708
Drop 3 accuracy: 6.393
Drop 3 loss: 111.080
Drop 3 accuracy: 8.118
Drop 3 loss: 25.219
Drop 3 accuracy: 8.106
Drop 3 loss: 26.634
Drop 3 accuracy: 3.512
Drop 3 loss: 26.916
Drop 3 accuracy: 2.047
Drop 3 loss: 20.891


/tmp/ipykernel_234104/1956587501.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df[df['exp_type']=='important'].groupby('exp_num').mean()
/tmp/ipykernel_234104/1956587501.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df[df['exp_type']=='important'].groupby('exp_num').mean()
/tmp/ipykernel_234104/1956587501.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df[df['exp_type']=='important'].gro

,model,drop_acc,increase_loss,drop_k,base_acc,k_acc,base_loss,k_loss
0,ShuffleNet(CE),3.325582,22.893906,3,0.957756,0.925905,0.260112,0.319661
1,ShuffleNet(QR),8.157610,95.766512,3,0.945206,0.868100,0.288708,0.565194
2,ResNet9(CE),1.553384,36.708291,3,0.960291,0.945374,0.189632,0.259242
3,ResNet9(QR),6.393184,111.080478,3,0.940318,0.880202,0.241312,0.509363
4,MLPMixer(CE),8.117946,25.219011,3,0.919630,0.844975,0.350531,0.438931
5,MLPMixer(QR),8.105685,26.633796,3,0.921885,0.847160,0.396319,0.501873
6,SimpleViT(CE),3.512102,26.915633,3,0.922446,0.890048,0.248335,0.315176
7,SimpleViT(QR),2.046664,20.890970,3,0.923888,0.904979,0.245232,0.296464


In [33]:
gilon_gradientshap_lstm = pd.read_csv('dataset/benchmark/gilon_activity_drop/GradientShap_gilon_activity_lstm.csv')
gilon_featureablation_lstm = pd.read_csv('dataset/benchmark/gilon_activity_drop/FeatureAblation_gilon_activity_lstm.csv')
gilon_integratedgradients_lstm = pd.read_csv('dataset/benchmark/gilon_activity_drop/IntegratedGradients_gilon_activity_lstm.csv')
gilon_saliency_lstm = pd.read_csv('dataset/benchmark/gilon_activity_drop/Saliency_gilon_activity_lstm.csv')
gilon_shapleyvaluesampling = pd.read_csv('dataset/benchmark/gilon_activity_drop/ShapleyValueSampling_gilon_activity_lstm.csv')
gilon_fit_lstm = pd.read_csv('dataset/benchmark/gilon_activity_drop/FIT_gilon_activity_lstm.csv')
gilon_dyna_lstm = pd.read_csv('dataset/benchmark/gilon_activity_drop/DynaMask_gilon_activity_lstm.csv')

gilon_gradientshap_tcn = pd.read_csv('dataset/benchmark/gilon_activity_drop/GradientShap_gilon_activity_tcn.csv')
gilon_featureablation_tcn = pd.read_csv('dataset/benchmark/gilon_activity_drop/FeatureAblation_gilon_activity_tcn.csv')
gilon_integratedgradients_tcn = pd.read_csv('dataset/benchmark/gilon_activity_drop/IntegratedGradients_gilon_activity_tcn.csv')
gilon_saliency_tcn = pd.read_csv('dataset/benchmark/gilon_activity_drop/Saliency_gilon_activity_tcn.csv')
gilon_shapleyvaluesampling_tcn = pd.read_csv('dataset/benchmark/gilon_activity_drop/ShapleyValueSampling_gilon_activity_tcn.csv')
gilon_dyna_tcn = pd.read_csv('dataset/benchmark/gilon_activity_drop/DynaMask_gilon_activity_tcn.csv')

gilon_laxcat = pd.read_csv('dataset/benchmark/gilon_activity_drop/laxcat_gilon_activity_laxcat.csv')

all_baseline_results = [[gilon_gradientshap_lstm, 'GradientShap(LSTM)'],
                        [gilon_featureablation_lstm, 'FeatureAblation(LSTM)'],
                        [gilon_integratedgradients_lstm, 'IntegratedGradients(LSTM)'],
                        [gilon_saliency_lstm, 'Saliency(LSTM)'],
                        [gilon_shapleyvaluesampling, 'ShapleyValueSampling(LSTM)'],
                        [gilon_gradientshap_tcn, 'GradientShap(TCN)'],
                        [gilon_featureablation_tcn, 'FeatureAblation(TCN)'],
                        [gilon_integratedgradients_tcn, 'IntegratedGradients(TCN)'],
                        [gilon_saliency_tcn, 'Saliency(TCN)'],
                        [gilon_shapleyvaluesampling_tcn, 'ShapleyValueSampling(TCN)'],
                        [gilon_laxcat, 'LaxCAT'],
                        [gilon_fit_lstm, 'FIT(LSTM)'],
                        [gilon_dyna_lstm, 'DynaMask(LSTM)'],
                        [gilon_dyna_tcn, 'DynaMask(TCN)']]

In [34]:
gilon_fit_lstm

,test_loss,test_accuracy,cv_num,exp_num,model_name,task_name,project_name,featuer_len,explainer_name,exp_type,feature_order
0,0.169136,0.949296,0,1001,lstm,gilon_activity,gilon_activity_lstm_EXP1001,14.0,base,base,0.0
1,0.261316,0.933749,1,1001,lstm,gilon_activity,gilon_activity_lstm_EXP1001,14.0,base,base,0.0
2,0.196182,0.950697,2,1001,lstm,gilon_activity,gilon_activity_lstm_EXP1001,14.0,base,base,0.0
3,0.151159,0.966454,3,1001,lstm,gilon_activity,gilon_activity_lstm_EXP1001,14.0,base,base,0.0
4,0.152634,0.958190,4,1001,lstm,gilon_activity,gilon_activity_lstm_EXP1001,14.0,base,base,0.0
...,...,...,...,...,...,...,...,...,...,...,...
130,0.491033,0.774214,0,1942,lstm,gilon_activity,gilon_activity_lstm_EXP1942,1.0,FIT,unimportant,13.0
131,0.485436,0.780727,1,1942,lstm,gilon_activity,gilon_activity_lstm_EXP1942,1.0,FIT,unimportant,13.0
132,0.504884,0.760627,2,1942,lstm,gilon_activity,gilon_activity_lstm_EXP1942,1.0,FIT,unimportant,13.0
133,0.514065,0.768471,3,1942,lstm,gilon_activity,gilon_activity_lstm_EXP1942,1.0,FIT,unimportant,13.0


In [35]:
def calculate_drop_acc_test_baselines(df, drop_k=2):
    base_df = df[df['exp_type']=='base'].groupby('exp_num').mean()
    important_df = df[df['exp_type']=='important'].groupby('exp_num').mean()

    base_acc = base_df[base_df['feature_order']==0]['test_accuracy'].mean()
    k_acc = important_df[important_df['feature_order']==drop_k]['test_accuracy'].mean()

    base_loss = base_df[base_df['feature_order']==0]['test_loss'].mean()
    k_loss = important_df[important_df['feature_order']==drop_k]['test_loss'].mean()
    # calculate as percentage
    drop_acc = (base_acc - k_acc) / base_acc *100
    drop_loss = (k_loss - base_loss) / base_loss *100
    print(f"Drop {drop_k} accuracy: {drop_acc:.3f}")
    print(f"Drop {drop_k} loss: {drop_loss:.3f}")

    # calculate drop weighted accuracy
    accuracies = important_df[important_df['feature_order']!=0]['test_accuracy'].values
    dacc_values = [base_acc - acc for acc in accuracies]
    weights_rev = [1/len(dacc_values) * i for i in range(len(dacc_values), 0, -1)]
    weighted_sum = sum([dacc * weight for dacc, weight in zip(dacc_values, weights_rev)])
        

    return [drop_acc, drop_loss, drop_k, base_acc, k_acc, base_loss, k_loss, weighted_sum]

In [36]:
all_drop_results_baselines = []
for result in all_baseline_results:
    drop_results = calculate_drop_acc_test_baselines(result[0], drop_k=3)
    all_drop_results_baselines.append([result[1], drop_results[0], drop_results[1], drop_results[2], drop_results[3], drop_results[4], drop_results[5], drop_results[6], drop_results[7]])

all_drop_results_baselines_df = pd.DataFrame(all_drop_results_baselines, columns=['model', 'drop_acc', 'increase_loss', 'drop_k', 'base_acc', 'k_acc', 'base_loss', 'k_loss','weighted_acc_drop'])
all_drop_results_baselines_df

Drop 3 accuracy: 1.404
Drop 3 loss: 21.223
Drop 3 accuracy: -1.857
Drop 3 loss: -33.197
Drop 3 accuracy: 1.404
Drop 3 loss: 21.223
Drop 3 accuracy: 0.283
Drop 3 loss: 7.424
Drop 3 accuracy: -0.278
Drop 3 loss: -7.913
Drop 3 accuracy: 2.209
Drop 3 loss: 2.445
Drop 3 accuracy: -2.185
Drop 3 loss: -18.574
Drop 3 accuracy: 2.298
Drop 3 loss: 6.714
Drop 3 accuracy: 3.093
Drop 3 loss: 5.600
Drop 3 accuracy: -0.858
Drop 3 loss: -13.638
Drop 3 accuracy: -4.217
Drop 3 loss: -23.779
Drop 3 accuracy: 0.506
Drop 3 loss: 3.397
Drop 3 accuracy: -0.843
Drop 3 loss: -7.683
Drop 3 accuracy: -0.270
Drop 3 loss: -10.275


/tmp/ipykernel_234104/2952072770.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  base_df = df[df['exp_type']=='base'].groupby('exp_num').mean()
/tmp/ipykernel_234104/2952072770.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  important_df = df[df['exp_type']=='important'].groupby('exp_num').mean()
/tmp/ipykernel_234104/2952072770.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  base_df = df[df['exp_type']=='

,model,drop_acc,increase_loss,drop_k,base_acc,k_acc,base_loss,k_loss,weighted_acc_drop
0,GradientShap(LSTM),1.404077,21.222575,3,0.951677,0.938315,0.186085,0.225578,0.140738
1,FeatureAblation(LSTM),-1.857385,-33.197350,3,0.951677,0.969354,0.186085,0.124310,0.114881
2,IntegratedGradients(LSTM),1.404077,21.222575,3,0.951677,0.938315,0.186085,0.225578,0.134591
3,Saliency(LSTM),0.282581,7.424412,3,0.951677,0.948988,0.186085,0.199901,0.009920
4,ShapleyValueSampling(LSTM),-0.278166,-7.913153,3,0.951677,0.954325,0.186085,0.171360,0.245066
5,GradientShap(TCN),2.208669,2.444686,3,0.927782,0.907290,0.247964,0.254026,0.225820
6,FeatureAblation(TCN),-2.184514,-18.574263,3,0.927782,0.948050,0.247964,0.201907,0.026665
7,IntegratedGradients(TCN),2.297740,6.714250,3,0.927782,0.906464,0.247964,0.264613,0.239909
8,Saliency(TCN),3.093344,5.599675,3,0.927782,0.899083,0.247964,0.261849,0.247645
9,ShapleyValueSampling(TCN),-0.857501,-13.638157,3,0.927782,0.935738,0.247964,0.214146,0.077919


# MS

In [37]:
microsoft_main_dir = 'dataset/importance_result_files_mine/microsoft_activity_new/'
shufflenet_ms = glob.glob(microsoft_main_dir + 'shufflenet/*')
resnet_ms = glob.glob(microsoft_main_dir + 'resnet9/*')
mlpmixer_ms = glob.glob(microsoft_main_dir + 'mlpmixer/*')
simplevit_ms = glob.glob(microsoft_main_dir + 'simplevit/*')

shufflenet_ms_df = read_files(shufflenet_ms)
resnet_ms_df = read_files(resnet_ms)
mlpmixer_ms_df = read_files(mlpmixer_ms)
simplevit_ms_df = read_files(simplevit_ms)

Total files: 4
Total files: 4
Total files: 4
Total files: 4


In [38]:
shufflenet_ms_qr_true = shufflenet_ms_df[shufflenet_ms_df['opl_loss']==True]
shufflenet_ms_qr_false = shufflenet_ms_df[shufflenet_ms_df['opl_loss']==False]
resnet_ms_qr_true = resnet_ms_df[resnet_ms_df['opl_loss']==True]
resnet_ms_qr_false = resnet_ms_df[resnet_ms_df['opl_loss']==False]
mlpmixer_ms_qr_true = mlpmixer_ms_df[mlpmixer_ms_df['opl_loss']==True]
mlpmixer_ms_qr_false = mlpmixer_ms_df[mlpmixer_ms_df['opl_loss']==False]
simplevit_ms_qr_true = simplevit_ms_df[simplevit_ms_df['opl_loss']==True]
simplevit_ms_qr_false = simplevit_ms_df[simplevit_ms_df['opl_loss']==False]

all_results_ms = [[shufflenet_ms_qr_false, 'ShuffleNet(CE)'],
                    [shufflenet_ms_qr_true, 'ShuffleNet(QR)'],
                    [resnet_ms_qr_false, 'ResNet9(CE)'],
                    [resnet_ms_qr_true, 'ResNet9(QR)'],
                    [mlpmixer_ms_qr_false, 'MLPMixer(CE)'],
                    [mlpmixer_ms_qr_true, 'MLPMixer(QR)'],
                    [simplevit_ms_qr_false, 'SimpleViT(CE)'],
                    [simplevit_ms_qr_true, 'SimpleViT(QR)']]


In [39]:
all_drop_results_ms = []
for result in all_results_ms:
    drop_results = calculate_drop_acc_test(result[0], drop_k=2)
    all_drop_results_ms.append([result[1], drop_results[0], drop_results[1], drop_results[2], drop_results[3], drop_results[4], drop_results[5], drop_results[6]])
all_drop_results_ms_df = pd.DataFrame(all_drop_results_ms, columns=['model', 'drop_acc', 'increase_loss', 'drop_k', 'base_acc', 'k_acc', 'base_loss', 'k_loss'])
all_drop_results_ms_df

Drop 2 accuracy: 7.782
Drop 2 loss: 2.907
Drop 2 accuracy: 21.033
Drop 2 loss: 53.704
Drop 2 accuracy: -0.667
Drop 2 loss: 5.823
Drop 2 accuracy: 1.267
Drop 2 loss: -1.893
Drop 2 accuracy: 0.731
Drop 2 loss: 1.449
Drop 2 accuracy: 6.093
Drop 2 loss: 7.008
Drop 2 accuracy: -1.110
Drop 2 loss: -12.429
Drop 2 accuracy: 10.963
Drop 2 loss: 6.655


/tmp/ipykernel_234104/1956587501.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df[df['exp_type']=='important'].groupby('exp_num').mean()
/tmp/ipykernel_234104/1956587501.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df[df['exp_type']=='important'].groupby('exp_num').mean()
/tmp/ipykernel_234104/1956587501.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df[df['exp_type']=='important'].gro

,model,drop_acc,increase_loss,drop_k,base_acc,k_acc,base_loss,k_loss
0,ShuffleNet(CE),7.781907,2.906541,2,0.845814,0.779994,1.087605,1.119217
1,ShuffleNet(QR),21.032605,53.704170,2,0.810094,0.639710,0.938707,1.442831
2,ResNet9(CE),-0.667042,5.823232,2,0.752070,0.757087,0.932356,0.986649
3,ResNet9(QR),1.266697,-1.893103,2,0.787307,0.777335,0.774118,0.759463
4,MLPMixer(CE),0.730946,1.449314,2,0.735932,0.730553,0.925499,0.938912
5,MLPMixer(QR),6.093399,7.008451,2,0.726080,0.681837,0.940260,1.006158
6,SimpleViT(CE),-1.109582,-12.429430,2,0.702690,0.710487,0.943790,0.826482
7,SimpleViT(QR),10.962773,6.654984,2,0.706256,0.628830,0.946512,1.009503


In [40]:
ms_gradientshap_lstm = pd.read_csv('dataset/benchmark/microsoft_activity_drop/GradientShap_microsoft_activity_new_lstm.csv')
ms_featureablation_lstm = pd.read_csv('dataset/benchmark/microsoft_activity_drop/FeatureAblation_microsoft_activity_new_lstm.csv')
ms_integratedgradients_lstm = pd.read_csv('dataset/benchmark/microsoft_activity_drop/IntegratedGradients_microsoft_activity_new_lstm.csv')
ms_saliency_lstm = pd.read_csv('dataset/benchmark/microsoft_activity_drop/Saliency_microsoft_activity_new_lstm.csv')
ms_shapleyvalues_lstm = pd.read_csv('dataset/benchmark/microsoft_activity_drop/ShapleyValueSampling_microsoft_activity_new_lstm.csv')
ms_fit_lstm = pd.read_csv('dataset/benchmark/microsoft_activity_drop/FIT_microsoft_activity_new_lstm.csv')
ms_dyna_lstm = pd.read_csv('dataset/benchmark/microsoft_activity_drop/DynaMask_microsoft_activity_new_lstm.csv')
ms_gradientshap_tcn = pd.read_csv('dataset/benchmark/microsoft_activity_drop/GradientShap_microsoft_activity_new_tcn.csv')
ms_featureablation_tcn = pd.read_csv('dataset/benchmark/microsoft_activity_drop/FeatureAblation_microsoft_activity_new_tcn.csv')
ms_integratedgradients_tcn = pd.read_csv('dataset/benchmark/microsoft_activity_drop/IntegratedGradients_microsoft_activity_new_tcn.csv')
ms_saliency_tcn = pd.read_csv('dataset/benchmark/microsoft_activity_drop/Saliency_microsoft_activity_new_tcn.csv')
ms_shapleyvalues_tcn = pd.read_csv('dataset/benchmark/microsoft_activity_drop/ShapleyValueSampling_microsoft_activity_new_tcn.csv')
ms_dyna_tcn = pd.read_csv('dataset/benchmark/microsoft_activity_drop/DynaMask_microsoft_activity_new_tcn.csv')
ms_laxcat_laxcat = pd.read_csv('dataset/benchmark/microsoft_activity_drop/laxcat_microsoft_activity_new_laxcat.csv')


all_baseline_results_ms = [[ms_gradientshap_lstm, 'GradientShap(LSTM)'],
                            [ms_featureablation_lstm, 'FeatureAblation(LSTM)'],
                            [ms_integratedgradients_lstm, 'IntegratedGradients(LSTM)'],
                            [ms_saliency_lstm, 'Saliency(LSTM)'],
                            [ms_shapleyvalues_lstm, 'ShapleyValueSampling(LSTM)'],

                            [ms_gradientshap_tcn, 'GradientShap(TCN)'],
                            [ms_featureablation_tcn, 'FeatureAblation(TCN)'],
                            [ms_integratedgradients_tcn, 'IntegratedGradients(TCN)'],
                            [ms_saliency_tcn, 'Saliency(TCN)'],
                            [ms_shapleyvalues_tcn, 'ShapleyValueSampling(TCN)'],
                            [ms_laxcat_laxcat, 'LaxCAT'],
                            [ms_fit_lstm, 'FIT(LSTM)'],
                            [ms_dyna_lstm, 'DynaMask(LSTM)'],
                            [ms_dyna_tcn, 'DynaMask(TCN)']]

In [41]:
all_drop_results_baselines_ms = []
for result in all_baseline_results_ms:
    drop_results = calculate_drop_acc_test_baselines(result[0], drop_k=1)
    all_drop_results_baselines_ms.append([result[1], drop_results[0], drop_results[1], drop_results[2], drop_results[3], drop_results[4], drop_results[5], drop_results[6], drop_results[7]])

all_drop_results_baselines_ms_df = pd.DataFrame(all_drop_results_baselines_ms, columns=['model', 'drop_acc', 'increase_loss', 'drop_k', 'base_acc', 'k_acc', 'base_loss', 'k_loss','weighted_acc_drop'])
all_drop_results_baselines_ms_df

Drop 1 accuracy: -3.356
Drop 1 loss: -25.408
Drop 1 accuracy: -3.356
Drop 1 loss: -25.408
Drop 1 accuracy: -3.356
Drop 1 loss: -25.408
Drop 1 accuracy: 4.517
Drop 1 loss: 12.589
Drop 1 accuracy: -3.356
Drop 1 loss: -25.408
Drop 1 accuracy: 2.513
Drop 1 loss: 18.236
Drop 1 accuracy: 1.927
Drop 1 loss: 11.776
Drop 1 accuracy: 0.764
Drop 1 loss: 13.076
Drop 1 accuracy: 3.054
Drop 1 loss: 20.301
Drop 1 accuracy: 1.653
Drop 1 loss: 9.848
Drop 1 accuracy: 1.003
Drop 1 loss: 0.701
Drop 1 accuracy: 1.655
Drop 1 loss: 8.036
Drop 1 accuracy: -2.127
Drop 1 loss: -3.728
Drop 1 accuracy: 4.270
Drop 1 loss: 28.377


/tmp/ipykernel_234104/2952072770.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  base_df = df[df['exp_type']=='base'].groupby('exp_num').mean()
/tmp/ipykernel_234104/2952072770.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  important_df = df[df['exp_type']=='important'].groupby('exp_num').mean()
/tmp/ipykernel_234104/2952072770.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  base_df = df[df['exp_type']=='

,model,drop_acc,increase_loss,drop_k,base_acc,k_acc,base_loss,k_loss,weighted_acc_drop
0,GradientShap(LSTM),-3.355554,-25.407954,1,0.806951,0.834028,0.920424,0.686563,-0.027199
1,FeatureAblation(LSTM),-3.355554,-25.407954,1,0.806951,0.834028,0.920424,0.686563,-0.036337
2,IntegratedGradients(LSTM),-3.355554,-25.407954,1,0.806951,0.834028,0.920424,0.686563,-0.027199
3,Saliency(LSTM),4.516516,12.589312,1,0.806951,0.770505,0.920424,1.036299,0.071139
4,ShapleyValueSampling(LSTM),-3.355554,-25.407954,1,0.806951,0.834028,0.920424,0.686563,-0.036337
5,GradientShap(TCN),2.513159,18.235779,1,0.815292,0.794802,0.801732,0.947934,0.057866
6,FeatureAblation(TCN),1.927496,11.776128,1,0.815292,0.799577,0.801732,0.896145,0.017359
7,IntegratedGradients(TCN),0.763585,13.076236,1,0.815292,0.809066,0.801732,0.906568,0.030305
8,Saliency(TCN),3.054341,20.301384,1,0.815292,0.790390,0.801732,0.964495,0.104696
9,ShapleyValueSampling(TCN),1.653199,9.847667,1,0.815292,0.801813,0.801732,0.880684,-0.007954


In [42]:
all_drop_results_baselines = []
for result in all_baseline_results:
    drop_results = calculate_drop_acc_test_baselines(result[0], drop_k=1)
    all_drop_results_baselines.append([result[1], drop_results[0], drop_results[1], drop_results[2], drop_results[3], drop_results[4], drop_results[5], drop_results[6]])

all_drop_results_baselines_df = pd.DataFrame(all_drop_results_baselines, columns=['model', 'drop_acc', 'increase_loss', 'drop_k', 'base_acc', 'k_acc', 'base_loss', 'k_loss'])
all_drop_results_baselines_df

Drop 1 accuracy: -0.717
Drop 1 loss: -7.243
Drop 1 accuracy: -0.717
Drop 1 loss: -7.243
Drop 1 accuracy: -0.717
Drop 1 loss: -7.243
Drop 1 accuracy: -1.067
Drop 1 loss: -23.899
Drop 1 accuracy: -0.717
Drop 1 loss: -7.243
Drop 1 accuracy: -1.861
Drop 1 loss: -30.127
Drop 1 accuracy: -1.478
Drop 1 loss: -25.785
Drop 1 accuracy: -2.476
Drop 1 loss: -36.140
Drop 1 accuracy: -0.189
Drop 1 loss: -17.368
Drop 1 accuracy: -1.750
Drop 1 loss: -15.880
Drop 1 accuracy: -2.456
Drop 1 loss: -17.093
Drop 1 accuracy: -0.546
Drop 1 loss: 0.102
Drop 1 accuracy: -0.374
Drop 1 loss: -1.960
Drop 1 accuracy: -1.363
Drop 1 loss: -22.742


/tmp/ipykernel_234104/2952072770.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  base_df = df[df['exp_type']=='base'].groupby('exp_num').mean()
/tmp/ipykernel_234104/2952072770.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  important_df = df[df['exp_type']=='important'].groupby('exp_num').mean()
/tmp/ipykernel_234104/2952072770.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  base_df = df[df['exp_type']=='

,model,drop_acc,increase_loss,drop_k,base_acc,k_acc,base_loss,k_loss
0,GradientShap(LSTM),-0.716756,-7.242949,1,0.951677,0.958498,0.186085,0.172607
1,FeatureAblation(LSTM),-0.716756,-7.242949,1,0.951677,0.958498,0.186085,0.172607
2,IntegratedGradients(LSTM),-0.716756,-7.242949,1,0.951677,0.958498,0.186085,0.172607
3,Saliency(LSTM),-1.067040,-23.898948,1,0.951677,0.961832,0.186085,0.141613
4,ShapleyValueSampling(LSTM),-0.716756,-7.242949,1,0.951677,0.958498,0.186085,0.172607
5,GradientShap(TCN),-1.861441,-30.127031,1,0.927782,0.945052,0.247964,0.173260
6,FeatureAblation(TCN),-1.477981,-25.785425,1,0.927782,0.941495,0.247964,0.184026
7,IntegratedGradients(TCN),-2.475883,-36.139556,1,0.927782,0.950753,0.247964,0.158351
8,Saliency(TCN),-0.188711,-17.368060,1,0.927782,0.929533,0.247964,0.204898
9,ShapleyValueSampling(TCN),-1.749724,-15.880298,1,0.927782,0.944016,0.247964,0.208587
